In [ ]:
from __future__ import print_function
try:
	raw_input
except:
	raw_input = input

import timeit
from concurrent import futures
from concurrent.futures import ThreadPoolExecutor


from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

import tensorflow_datasets as tfds
from tensorflow.keras.datasets import mnist, cifar10, cifar100

from tensorflow.keras.datasets import mnist, cifar10, cifar100
import tensorflow.keras.layers as L
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation
from keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy



tfds.disable_progress_bar()

import matplotlib.pyplot as plt
import numpy as np



import numpy as np
import time
import datetime
import os
from PIL import Image
import sys
from keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

RESNET_MEAN = np.array([103.939, 116.779, 123.68])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



![](https://drive.google.com/uc?export=view&id=1YMexys51Bbmkb-JDdFF1AiKWOpot-SEE)


In [ ]:
DATASET = 'mnist'

In [ ]:
# Utility functions
def print_shapes(x_train, x_test, y_train, y_test):
  print(f"x_train: {x_train.shape}\n"\
      f"x_test: {x_test.shape}\n"\
      f"y_train: {y_train.shape}\n"\
      f"y_test: {y_test.shape}\n")

In [ ]:
# loading the dataset
if DATASET == 'mnist':
  (x_train, y_train), (x_test, y_test) = mnist.load_data()
else:
  (x_train, y_train), (x_test, y_test) = cifar10.load_data()
  
print_shapes(x_train, x_test, y_train, y_test)

11501568/11490434 [==============================] - 0s 0us/step
x_train: (60000, 28, 28)
x_test: (10000, 28, 28)
y_train: (60000,)
y_test: (10000,)



In [ ]:
# MNIST
labels = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']

In [ ]:
# Pre-process data
if DATASET == 'mnist':
  img_rows, img_cols, channels =  28, 28, 1 
else:
  img_rows, img_cols, channels =  32, 32, 3

num_classes = len(labels)

x_train = x_train / 255
x_test = x_test / 255

x_train = x_train.reshape((-1, img_rows, img_cols, channels))
x_test = x_test.reshape((-1, img_rows, img_cols, channels))

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

print("Data shapes", x_test.shape, y_test.shape, x_train.shape, y_train.shape)

Data shapes (10000, 28, 28, 1) (10000, 10) (60000, 28, 28, 1) (60000, 10)


In [ ]:
folder_id = time.strftime('[%d%m%Y]_%H%M%s', time.localtime())
input_path = "/content/drive/MyDrive/Thesis_notebooks/keras_boundary_attack/images"
os.mkdir(os.path.join(input_path, folder_id))

In [ ]:
def process_img(image,adversarial_label,squeeze = False, folder = folder_id):
  """Export image file."""
  label = adversarial_label
  print("Labeling from function",label)
  
  cmap = plt.cm.jet
  id_no = time.strftime('%Y%m%d_%H%M%S', datetime.datetime.now().timetuple())
  if DATASET == 'mnist':
    norm = plt.Normalize(vmin=image.reshape((img_rows, img_cols)).min(), vmax=image.reshape((img_rows, img_cols)).max())
    # map the normalized data to colors
    # image is now RGBA (512x512x4) 
    _image = cmap(norm(image.reshape((img_rows, img_cols))))
    # Save with predicted label for image (may not be adversarial due to uint8 conversion)
    plt.imsave(os.path.join("/content/drive/MyDrive/Thesis_notebooks/keras_boundary_attack/images", folder,"{}_{}.png".format(id_no, label)), _image)

  else:
    plt.figure(figsize=(3,3))
    _image = (1/(2*1.050000000745058)) * image + 0.35
    print(np.max(image))
    plt.imsave(os.path.join("/content/drive/MyDrive/Thesis_notebooks/keras_boundary_attack/images", folder,"{}_{}.png".format(id_no, label)),_image.reshape(img_rows, img_cols, channels))

In [ ]:
import random

rand_idx = 2#random.randint(0,49999)
_image = x_train[rand_idx]
image_label = y_train[rand_idx]
print(rand_idx)
if DATASET == 'mnist':
  process_img(_image.reshape((img_rows, img_cols, channels)), str(4))
else:
  process_img(_image,image_label)


2
Labeling from function 4


In [ ]:
def sphere_perturbation(delta, prev_sample, target_sample):
    """Generate orthogonal perturbation."""
    perturb = np.random.randn(1, 224, 224, 3)
    perturb /= np.linalg.norm(perturb, axis=(1, 2))
    perturb *= delta * np.mean(get_diff(target_sample, prev_sample))
    # Project perturbation onto sphere around target
    # Orthorgonal vector to sphere surface
    diff = (target_sample - prev_sample).astype(np.float32)
    diff /= get_diff(target_sample, prev_sample)  # Orthogonal unit vector
    # We project onto the orthogonal then subtract from perturb
    # to get projection onto sphere surface
    perturb -= (np.vdot(perturb, diff) / np.linalg.norm(diff)**2) * diff
    # Check overflow and underflow
    overflow = (prev_sample + perturb) - 255 + RESNET_MEAN
    perturb -= overflow * (overflow > 0)
    underflow = -RESNET_MEAN
    perturb += underflow * (underflow > 0)
    return perturb

In [ ]:
from PIL import Image as im

def forward_perturbation(epsilon, prev_sample, target_sample):
    """Generate forward perturbation."""
    perturb = (target_sample - prev_sample).astype(np.float32)
    perturb /= get_diff(target_sample, prev_sample)
    perturb *= epsilon
    return perturb


def get_converted_prediction(sample, classifier):
    """
    The loss of precision often causes the label of the image to change, particularly
    because we are very close to the boundary of the two classes.
    This function checks for the label of the exported sample
    by simulating the export process.
    """
    #sample = (sample + RESNET_MEAN).astype(np.uint8).astype(np.float32) - RESNET_MEAN
    label = decode_predictions(classifier.predict(sample), top=1)[0][0][1]
    return label


def save_image(sample, classifier, folder):
    """Export image file."""
    label = get_converted_prediction(np.copy(sample), classifier)
    print("Labeling from function",label)
    sample = sample.reshape(224, 224, 3)
    # Reverse preprocessing, see https://github.com/keras-team/keras/blob/master/keras/applications/imagenet_utils.py
    mean = [103.939, 116.779, 123.68]
    sample[..., 0] += mean[0]
    sample[..., 1] += mean[1]
    sample[..., 2] += mean[2]
    sample = sample[..., ::-1].astype(np.uint8)
    # Convert array to image and save
    sample = Image.fromarray(sample)
    id_no = time.strftime('%Y%m%d_%H%M%S', datetime.datetime.now().timetuple())
    # Save with predicted label for image (may not be adversarial due to uint8 conversion)
    sample.save(os.path.join("/content/drive/MyDrive/Thesis_notebooks/my_boundary_attack/boundary-attack-master/images", folder,"{}_{}.png".format(id_no, label)))

#preprocessing with keras
def preprocess(sample_path, mnist_flg = True):
    """Load and preprocess image file."""
    

    img = image.load_img(sample_path, target_size=(224, 224))
  
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

#keep mean error for each iteration
def get_diff(sample_1, sample_2):
    """Channel-wise norm of difference between samples."""
    sample_1 = sample_1.reshape(3, 224, 224)
    sample_2 = sample_2.reshape(3, 224, 224)
    diff = []
    for i, channel in enumerate(sample_1):
      diff.append(np.linalg.norm((channel - sample_2[i]).astype(np.float32)))
    return np.array(diff)
    #return np.linalg.norm(sample_1 - sample_2, axis=(1, 2))

In [ ]:
class MyCNN(Sequential):
  def __init__(self,input_size,filter_size=128, kernel_size = (3,3), dropout_flg = True, dropout_rate = 0.3):
    super(MyCNN, self).__init__()
    self.input_size = input_size
    self.filter_size = filter_size
    self.kernel_size = kernel_size
    self.dropout_flg = dropout_flg
    self.dropout_rate = dropout_rate

    self.add(L.Conv2D(self.filter_size, kernel_size=self.kernel_size,padding='same', activation='relu', input_shape=self.input_size))
    if dropout_flg:
      self.add(L.Dropout(self.dropout_rate))
    self.add(L.MaxPooling2D(pool_size=(2, 2)))

    self.add(L.Conv2D(self.filter_size/2, kernel_size=self.kernel_size,padding='same', activation='relu',input_shape=self.input_size))
    if dropout_flg:
      self.add(L.Dropout(self.dropout_rate))
    self.add(L.MaxPooling2D(pool_size=(2, 2)))

    self.add(L.Conv2D(self.filter_size/2, kernel_size=self.kernel_size,padding='same', activation='relu'))
    if dropout_flg:
      self.add(L.Dropout(self.dropout_rate))
    self.add(L.MaxPooling2D(pool_size=(2, 2)))

    self.add(L.Conv2D(self.filter_size/2, kernel_size=self.kernel_size,padding='same', activation='relu'))
    self.add(L.MaxPooling2D(pool_size=(2, 2)))

    if dropout_flg:
      self.add(L.Dropout(self.dropout_rate))
      
    self.add(L.Flatten())
    self.add(L.Dense(32))
    self.add(L.Dropout(0.2))
    self.add(L.Dense(num_classes, activation='softmax'))

In [ ]:
from scipy.spatial import distance

class boundary_attack():
  def __init__(self,init_sample_path,target_sample_path,dir_path):
    #create ResNet Classifier (prefer resnet for better results)
    self.classifier = ResNet50(weights='imagenet') #can be changed to a CNN classifier
    #init paths and folders for the statistics
    self.initial_sample = preprocess(init_sample_path)
    self.target_sample = preprocess(target_sample_path)
    self.folder = time.strftime('[%d%m%Y]_%H%M%s', time.localtime())
    os.mkdir(os.path.join(dir_path, self.folder))
    save_image(np.copy(self.initial_sample), self.classifier, self.folder)
    #classifier initialised with our picture (in this case picture of seal that you can faintly see behind the eel when the attack is executed)
    self.attack_class = np.argmax(self.classifier.predict(self.initial_sample))
    #Our target image is the eel.The who,e point is to create an image that looks almost exactly like the eel but has pertrubations "inspired by the eel", 
    #so that it's misclassified
    self.target_class = np.argmax(self.classifier.predict(self.target_sample))

    self.adversarial_sample = self.initial_sample
    self.n_steps = 0
    self.n_calls = 0
    self.epsilon = 1.
    self.delta = 0.1
    self.prev_dist = 0
    self.repetitions = 0
    self.no_bound_found = 0
    self.boundaries_found = 0
    self.errors = []
  
  def _attack(self):
        # Move first step to the boundary
    # get first adversarial point to begin initiating the attack 
    #this is the hybrid version, where we keep looping, till we find a closer adversarial point
    while True:
        trial_sample = self.adversarial_sample + forward_perturbation(self.epsilon, self.adversarial_sample, self.target_sample) - self.prev_dist
        prediction = self.classifier.predict(trial_sample.reshape(1, 224, 224, 3))
        self.n_calls += 1
        dist = distance.euclidean(prediction, self.classifier.predict(self.target_sample)) 
   
        if self.repetitions == 0:
          self.prev_dist = dist
        if dist < self.prev_dist:
          print("FOUND CLOSER BOUNDARY")
          print(dist)
        self.prev_dist = dist
        if self.repetitions > 1000 or self.no_bound_found > 100:
          break
        if np.argmax(prediction) == self.attack_class:
            self.adversarial_sample = trial_sample
            print("Found another bound")
            self.boundaries_found += 1
            #self.epsilon *= 0.9
        else:
          print("Didn't Find bound")
          self.epsilon *= 0.9
          self.no_bound_found += 1
          break
        self.repetitions += 1

    # Iteratively run attack
    while True:
        print("Step #{}...".format(self.n_steps))
        # Orthogonal step
        print("\tDelta step...")
        d_step = 0
        while True:
            d_step += 1
            print("\t#{}".format(d_step))
            trial_samples = []
            for i in np.arange(10):
                trial_sample = self.adversarial_sample + sphere_perturbation(self.delta, self.adversarial_sample, self.target_sample)
                trial_samples.append(trial_sample)
            
            trial_samples = np.asarray(trial_samples).reshape((-1,224,224,3))
            predictions = self.classifier.predict(trial_samples)
            self.n_calls += 10
            predictions = np.argmax(predictions, axis=1)
            d_score = np.mean(predictions == self.attack_class)
            if d_score > 0.0:
                if d_score < 0.3:
                    self.delta *= 0.9
                elif d_score > 0.7:
                    self.delta /= 0.9
                self.adversarial_sample = np.array(trial_samples)[np.where(predictions == self.attack_class)[0][0]]
                break
            else:
                self.delta *= 0.9
        # Forward step
        print("\tEpsilon step...")
        e_step = 0
        while True:
            e_step += 1
            print("\t#{}".format(e_step))
            trial_sample = self.adversarial_sample + forward_perturbation(self.epsilon, self.adversarial_sample, self.target_sample)
            prediction = self.classifier.predict(trial_sample)
            self.n_calls += 1
            #adversarial step
            if np.argmax(prediction) == self.attack_class:
                temp = get_converted_prediction(np.copy(trial_sample), self.classifier)
                print(temp)
                self.adversarial_sample = trial_sample
                self.epsilon /= 0.5
                break
            elif e_step > 500:
                break
            else:
                self.epsilon *= 0.5

        self.n_steps += 1
        # keep some checkpoints so that we save the pictures whilst the attack progresses
        chkpts = [250, 750]
        if self.n_steps % 10 == 0:# in chkpts:
            print("{} steps".format(self.n_steps))
            save_image(np.copy(self.adversarial_sample), self.classifier, self.folder)
        diff = np.mean(get_diff(self.adversarial_sample, self.target_sample))
        self.errors.append(diff)
        #stop at 1000 steps ~= 40 minutes
        if diff <= 1e-3 or e_step > 500 or self.n_steps > 25000:
            print("{} steps".format(self.n_steps))
            print("Mean Squared Error: {}".format(diff))
            save_image(np.copy(self.adversarial_sample), self.classifier, self.folder)
            break

        print("Mean Squared Error: {}".format(diff))
        print("Calls: {}".format(self.n_calls))
        print("Attack Class: {}".format(self.attack_class))
        print("Target Class: {}".format(self.target_class))
        print("Adversarial Class: {}".format(np.argmax(prediction)))

In [ ]:
import datetime
init_path = '/content/drive/MyDrive/Thesis_notebooks/keras_boundary_attack/images/2.png'#'/content/drive/MyDrive/Thesis_notebooks/my_boundary_attack/boundary-attack-master/images/original/awkward_moment_seal.png'
target_path = '/content/drive/MyDrive/Thesis_notebooks/keras_boundary_attack/images/4.png' #='/content/drive/MyDrive/Thesis_notebooks/my_boundary_attack/boundary-attack-master/images/original/bad_joke_eel.png'
_dir_path = "/content/drive/MyDrive/Thesis_notebooks/my_boundary_attack/boundary-attack-master/images"
model = boundary_attack(init_path,target_path,_dir_path)

49152/35363 [=========================================] - 0s 0us/step
Labeling from function analog_clock


In [ ]:
model._attack()

Streaming output truncated to the last 5000 lines.
Mean Squared Error: 417.3822326660156
Calls: 134265
Attack Class: 409
Target Class: 872
Adversarial Class: 409
Step #11021...
	Delta step...
	#1
	Epsilon step...
	#1
	#2
	#3
analog_clock
Mean Squared Error: 416.94580078125
Calls: 134278
Attack Class: 409
Target Class: 872
Adversarial Class: 409
Step #11022...
	Delta step...
	#1
	Epsilon step...
	#1
analog_clock
Mean Squared Error: 414.7092590332031
Calls: 134289
Attack Class: 409
Target Class: 872
Adversarial Class: 409
Step #11023...
	Delta step...
	#1
	Epsilon step...
	#1
analog_clock
Mean Squared Error: 408.8643798828125
Calls: 134300
Attack Class: 409
Target Class: 872
Adversarial Class: 409
Step #11024...
	Delta step...
	#1
	Epsilon step...
	#1
	#2
	#3
	#4
	#5
	#6
analog_clock
Mean Squared Error: 409.7511291503906
Calls: 134316
Attack Class: 409
Target Class: 872
Adversarial Class: 409
Step #11025...
	Delta step...
	#1
	Epsilon step...
	#1
analog_clock
Mean Squared Error: 410.1913

In [ ]:
import matplotlib.pyplot as plt

x_data = list(range(0, len(model.errors)))
y_data = model.errors

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

ax.plot(x_data, y_data)
ax.set_xlabel('steps to complete')
ax.set_ylabel('mse')

plt.show()